Task 2: Implementing a CNN<br>
MNIST<br>
1. Build a Simple CNN on the MNIST Dataset:<br>
   - Create a simple CNN architecture on the MNIST dataset and document what difference have you seen in results compared to your previous assignment where you had to develop a simple ANN model on the MNIST dataset.<br>
   - Document the architecture of your CNN, including the number of layers, types of layers, and parameters used.<br>
   - Train your CNN on the MNIST dataset and explore the use of callbacks (e.g., EarlyStopping, ModelCheckpoint) while training models.<br>
   - Document the training process, including data preprocessing steps, training parameters (learning rate, batch size, epochs), and any challenges faced.<br>
   - Evaluate the performance  <br>


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the images to the range [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Reshape images to (28, 28, 1) for the CNN
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# One-hot encode the labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Build the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)

# Train the model
history = model.fit(train_images, train_labels,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping, model_checkpoint])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.4f}")

# Load the best model (optional)
best_model = models.load_model('best_model.keras')

# Evaluate the best model on the test set (optional)
best_test_loss, best_test_acc = best_model.evaluate(test_images, test_labels)
print(f"Best model test accuracy: {best_test_acc:.4f}")


2. Build a Simple CNN for a Binary Classification Task:<br>
   - Create a simple CNN architecture on the Cat-vs-Dog dataset and document<br>   
   - Document the architecture of your CNN, including the number of layers, types of layers, and parameters used.<br>
   - Train your CNN on the Cat-vs-Dog dataset and explore the use of callbacks (e.g., EarlyStopping, ModelCheckpoint) while training models.<br>
   - Document the training process, including data preprocessing steps, training parameters (learning rate, batch size, epochs), and any challenges faced.<br>
   - Evaluate the performance<br>


In [ ]:
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the path to the dataset
data_dir = 'F:\\Bitwise ML DL Intership\\week 9\\Cat vs Dog\\PetImages'

# Function to check if an image can be opened
def is_image_valid(filepath):
    try:
        img = Image.open(filepath)
        img.verify()  # Verify that it is, in fact, an image
        return True
    except (IOError, SyntaxError) as e:
        print(f"Invalid file: {filepath}")
        return False

# Remove bad files from the dataset
for root, dirs, files in os.walk(data_dir):
    for file in files:
        filepath = os.path.join(root, file)
        if not is_image_valid(filepath):
            os.remove(filepath)

# Image data generators with augmentation for training and validation split
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)  # 20% of data for validation

# Loading data with the generators
train_generator = train_datagen.flow_from_directory(data_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(data_dir,
                                                         target_size=(150, 150),
                                                         batch_size=32,
                                                         class_mode='binary',
                                                         subset='validation')

# Build the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)

# Train the model
history = model.fit(train_generator,
                    epochs=6,
                    validation_data=validation_generator,
                    callbacks=[early_stopping, model_checkpoint])

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_acc:.4f}")

# Load the best model (optional)
best_model = models.load_model('best_model.keras')

# Evaluate the best model on the validation set (optional)
best_val_loss, best_val_acc = best_model.evaluate(validation_generator)
print(f"Best model validation accuracy: {best_val_acc:.4f}")
